# Exercitii `ipywidgets`:

Problema 1. (2 puncte) Definiti o functie polinomiala de gradul 3, $f:\mathbb{R} \rightarrow \mathbb{R}$, cu coeficienti constanti prestabiliti. Aplicati algoritmul gradient descent pentru a vedea cum evolueaza cautarea minimului. Folositi minim doua controale ipywidgets: unul pentru pozitia initiala a lui $x$, altul pentru coeficientul $\alpha>0$ cu care se inmulteste gradientul. Gradientul va fi calculat analitic de voi sau folosind biblioteca [autograd](https://github.com/HIPS/autograd). 
Modificarea facuta prin metoda gradient descent este:
$$
x = x - \alpha \cdot f'(x)
$$
Se vor efectua minim 10 iteratii (optional: numarul de iteratii poate fi dat printr-un control ipywidgets), se vor marca pe grafic pozitiile succesive, in mod convenabil.

In [1]:
import ipywidgets as widgets
print(f'IPywidgets version: {widgets.__version__}')

import numpy as np
print(f'NumPy version: {np.__version__}')

import matplotlib.pyplot as plt
import matplotlib
print(f'Matplotlib version: {matplotlib.__version__}')

IPywidgets version: 7.5.1
NumPy version: 1.19.2
Matplotlib version: 3.3.2


In [29]:
from ipywidgets import interact, fixed
from typing import List
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import LinearNDInterpolator
from random import randint
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
from scipy.interpolate import Rbf
import pylab as py

In [3]:
f = lambda x: x ** 3 + 2 * x ** 2 + 4 *x - 2
f_prime = lambda x: 3 * x ** 2 + 4 * x + 4

xf: np.ndarray = np.linspace(-20, 20, 100)
yf: np.ndarray = f(xf)

def sgd_plot(xf, yf, res: List[float] = None, title: str = 'Functia $f(x)=x^3 + 4 \cdot x^2 + 9 \cdot x - 20$') -> None:
    """
    Functie care afiseaza graficul
    
    :param xf: np.ndarray care contine 100 de puncte in intervalul (-20,20), adica domeniul functiei
    :param yf: np.ndarray care contine valorile functiei in punctele din xf, codomeniul functiei
    :param res: o lista care poate sa contina mai multe valori, caz in care adauga si aceasta functie in grafic
    :param title: string ce contine titlul graficului(functiei)
    :returns: None
    """
    _, ax = plt.subplots(1, 1, figsize=(10, 5))
    ax.plot(xf, yf)
    if res is not None:
        ax.plot(res, [f(x) for x in res], 'o--')
    ax.set_title(title)
    ax.grid() ; plt.show()

def sgd(x: int, alpha: float, epochs: int) ->list:
    """
    Functie care cauta minimul
    
    :param x: int pentru pozitia initiala
    :param alpha: float pentru rata de invatare
    :param epochs: int pentru numarul de epoci(interatii)
    :returns: o lista de float-uri ce reprezinta traiectoria
    """
    res = [x]
    for _ in range(epochs):
        x -= alpha * f_prime(x)
        res += [x]
    return res

def SGD(x: int = -5, alpha: float = 0.00001, epochs: int = 10) -> None:
    """
    Functie care apeleaza functiile sgd si sgd_plot, facilitand interactiunea utilizatorului cu graficul pentru ca va fi folosita cu interact
    
    :param x: int pentru pozitia initiala
    :param alpha: float pentru rata de invatare
    :param epochs: int pentru numarul de epoci(interatii)
    :returns: None
    """
    res = sgd(x, alpha, epochs)
    sgd_plot(xf, yf, res, title=f'$f(x)=x^3 + 2 \cdot x^2 + 4 \cdot x - 2$, start x: {x}, alpha={alpha}')
    
interact(SGD, x = (-5,20), alpha = (0.00001, 0.01, 0.00001), epochs = (10, 50));



interactive(children=(IntSlider(value=-5, description='x', max=20, min=-5), FloatSlider(value=1e-05, descripti…

Problema 2. (3 puncte) Generati o lista de $n=100$ de perechi de valori $\{x_i, y_i\}_{i=0,n-1}$ in intervalul [-20, 10), afisati aceste valori pe un grafic, impreuna cu o dreapta definita de o functie liniara $y=a \cdot x+b$. Intr-un alt plot afisati, ca histograma, distanta dintre punctele de coordonate $(x_i, y_i)$ si punctele de intersectie ale verticalelor duse prin $x_i$ cu dreapta data, $\hat{y}_i$. Dreapta trebuie sa fie controlabila din widgets, prin cei doi coeficienti $a$ si $b$. Constatati modificarea histogramei in functie de pozitia dreptei si afisati mean squared error: $$MSE=\frac{1}{n} \cdot \sum_{i=0}^{n-1} (y_i - (a\cdot x_i + b))^2$$
*Indicatii:*
1. Pentru generare de valori distribuite uniform in intervalul [0, 1) puteti folosi functia [numy.random.uniform](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.uniform.html) si sa faceti inmultire si adunare in mod convenabil.
1. Puteti opta sa returnati cele $n$ puncte sub forma `vector_x`, `vector_y`.


In [4]:
from sklearn.metrics import mean_squared_error 

vector_x: np.ndarray = np.random.uniform(-20,10, 100)
vector_y: np.ndarray = np.random.uniform(-20,10, 100)

def linear_function(vector_x, vector_y, a: float, b: float, bins_nr: int = 100) -> None:
    """
    Functie care afiseaza graficul functiei a*x+b, 100 de puncte alese aleatoriu si realizeaza o histograma cu distanta dintre punctele alese si punctele de intersectie ale verticalelor duse pe graficul functiei.
    De asemenea, afiseaza si MSE(mean squared error).
    
    :param vector_x:  np.ndarray ce contine pozitiile pe axa x a punctelor alese aleatoriu
    :param vector_y:  np.ndarray ce contine pozitiile pe axa y a punctelor alese aleatoriu
    :param a: float ce reprezinta primul coeficient al functiei
    :param b: float ce reprezinta al doilea coeficient al functiei
    :param bins_nr: int ce reprezinta numarul de bins uri pe care dorim sa-l reprezentam in histograma
    :returns: None
    """
    f = lambda x: a* x + b
    xf: np.ndarray = np.linspace(-20, 10, 100)
    yf: np.ndarray = f(xf)
    
    plt.figure(figsize=(10, 5))
    plt.grid(axis='both')
    plt.axhline(y=0, color='k')
    plt.axvline(x=0, color='k')
    plt.xlabel('x')
    plt.ylabel(f'${a} \cdot x + {b}$')
    plt.plot(xf, yf, vector_x, vector_y, 'ro')
    plt.show()
    
    intersection_y: np.ndarray = f(vector_x)
    distances: np.ndarray = np.abs(vector_y - intersection_y)
    
    plt.figure(figsize=(10, 5))
    plt.xlabel('Distances')
    plt.ylabel('Occurrences')
    plt.hist(distances, bins = bins_nr)
    plt.show()
    
    MSE: float = np.square(np.subtract(yf, intersection_y)).mean() 
    print(MSE)
    print(mean_squared_error(yf, intersection_y))
    
interact(linear_function, vector_x = fixed(vector_x), vector_y = fixed(vector_y), a = (-5,5.0), b = (-5,5.0), bins_nr = (1,100));

interactive(children=(FloatSlider(value=0.0, description='a', max=5.0, min=-5.0), FloatSlider(value=0.0, descr…

Problema 3. (2 puncte) Incarcati fisierul iris.data din setul de date iris
(https://archive.ics.uci.edu/ml/datasets/iris). In functie de alegerile exprimate de un utilizator, afisati intr-un grafic
2D coloanele numerice alese (de exemplu, coloana 0 si coloana 2). Numele coloanelor se afla in fisierul
iris.names

In [28]:
def g(atribut1:int,atribut2:int)->None:
    
    """
    Functia afiseaza graficul 2D avand pe axele x si y atributele selectate
    :param atribut1: reprezinta valoarea de la cheia aleasa din meniul dropdown pentru axa x, tip int
    :param atribut2: reprezinta valoarea de la cheia aleasa din meniul dropdown pentru axa y, tip int
    :returns: None
    """
    data1:list = [values[atribut1], values[4]]
    d1= pd.concat(data1, axis=1)
    data2= [values[atribut2], values[4]]
    d2= pd.concat(data2, axis=1)
    
    col1:list=[]

    for i in d1[4]:
        if i =='Iris-setosa':
            col1.append('blue')
        elif i=="Iris-versicolor":
            col1.append('black')
        elif i=="Iris-virginica" :
            col1.append("pink")
    for i in range(len(d1[4])):
        plt.scatter(d1[atribut1][i],d2[atribut2][i],c=col1[i])
    plt.xlabel('atribut1')
    plt.ylabel('atribut2')
    plt.legend(["Iris-setosa","Iris-versicolor","Iris-virginica"], loc ="lower right") 
    ax = plt.gca()
    leg = ax.get_legend()
    leg.legendHandles[0].set_color('blue')
    leg.legendHandles[1].set_color('black')
    leg.legendHandles[2].set_color('pink')
    plt.show()
    
    
values=pd.read_csv("iris.data" , header=None)
interact(g, 
         atribut1 = {
    'sepal length in cm':0,
    'sepal width in cm' :1,
    'petal length in cm':2,
    'petal width in cm' :3
}, 
         atribut2 = {
    'sepal length in cm':0,
    'sepal width in cm' :1,
    'petal length in cm':2,
    'petal width in cm' :3
});


interactive(children=(Dropdown(description='atribut1', options={'sepal length in cm': 0, 'sepal width in cm': …

Problema 4 (3 puncte) Generati  𝑛  perechi de puncte aleatoare, folosind o functie  𝑓:ℝ→ℝ  de alease e voi (de exemplu: functie polinomiala + zgomot aleator adaugat). Alegeti 5 metode de interpolare din scipy.interpolate si reprezentati grafic valorile interpolate. Folositi controale ipywidgets cel putin pentru alegerea lui  𝑛  si a metodei de interpolare aleasa.

In [30]:
def func(xcoord:int, ycoord:int)->int:
    """
    Functia asupra careia se realizeaza interpolarea
    :param xcoord :valoarea de pe axa x
    :param ycoord : valoarea de pe axa y
    :returns: valoarea functiei
    """
    return xcoord**3+2*ycoord+1

def g(n:int,option:int)->None:
    """
    Functia afiseaza garficul functiei interpolate prin 5 metode din scipy.interpolate
    :param n : numar random folosit pentru dimensiunile array-urilor sau pentru valorile acestora
    :param option: reprezinta valoarea de la cheia aleasa din mediul dropdown , indica metoda de interpolare
    :returns: None
    """
    if option==0:
        points = np.random.rand(n, 2)
        grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]
        values = func(points[:,0], points[:,1])
        grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
        plt.imshow(grid_z0.T, extent=(0,1,0,1), origin='lower')
        plt.title('Nearest')
        plt.gcf().set_size_inches(6, 6)
        plt.show()
    elif option==1:
        x = np.random.uniform(-1,1,n) 
        y = np.random.uniform(-1,1,n)
        fvals=func(x,y)
        new_func=Rbf(x,y,fvals)
        xnew,ynew=np.mgrid[-1:1:100j,-1:1:100j]
        fnew=new_func(xnew,ynew)
        py.figure(1)
        py.clf()
        py.imshow(fnew,extent=[-1,1,-1,1],cmap=py.cm.jet)
        py.scatter(x,y,30,fvals,cmap=py.cm.jet)
            
    elif option==2:
        x = np.random.random(n) - 0.5
        y = np.random.random(n) - 0.5
        z = func(x, y)
        X = np.linspace(min(x), max(x))
        Y = np.linspace(min(y), max(y))
        X, Y = np.meshgrid(X, Y)  # 2D grid for interpolation
        interp = LinearNDInterpolator(list(zip(x, y)), z)
        Z = interp(X, Y)
        plt.pcolormesh(X, Y, Z, shading='auto')
        plt.plot(x, y,"ok")

        plt.show()
    elif option==3:
        x = np.random.random(n) - 0.5
        y = np.random.random(n) - 0.5
        z = func(x, y)
        X = np.linspace(min(x), max(x))
        Y = np.linspace(min(y), max(y))
        X, Y = np.meshgrid(X, Y)  # 2D grid for interpolation
        interp = NearestNDInterpolator(list(zip(x, y)), z)
        Z = interp(X, Y)
        plt.pcolormesh(X, Y, Z, shading='auto')
        plt.plot(x, y, "ok")
        plt.show()
    elif option==4:
        x = np.random.random(n) - 0.5
        y = np.random.random(n) - 0.5
        z = func(x, y)
        X = np.linspace(min(x), max(x))
        Y = np.linspace(min(y), max(y))
        X, Y = np.meshgrid(X, Y)  # 2D grid for interpolation
        interp = CloughTocher2DInterpolator(list(zip(x, y)), z)
        Z = interp(X, Y)
        plt.pcolormesh(X, Y, Z, shading='auto')
        plt.plot(x, y, "ok")
        plt.show()

   
    
interact(g,n=(5,1000),option={'Griddata ':0,'Rbf':1,'LinearNDInterpolator':2,"NearestNDInterpolator":3,'CloughTocher2DInterpolator':4});

interactive(children=(IntSlider(value=502, description='n', max=1000, min=5), Dropdown(description='option', o…